**Задание**

1.Взять датасет

https://huggingface.co/datasets/merionum/ru_paraphraser

решить задачу парафраза

In [1]:
!pip install transformers sentencepiece datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.2 MB/s 
     |████████████████████████████████| 1.3 MB 54.4 MB/s 
     |████████████████████████████████| 451 kB 73.3 MB/s 
     |████████████████████████████████| 182 kB 75.6 MB/s 
     |████████████████████████████████| 7.6 MB 45.2 MB/s 
     |████████████████████████████████| 212 kB 79.1 MB/s 
     |████████████████████████████████| 132 kB 75.2 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [14]:
from datasets import load_dataset
import torch
from transformers import AutoModelForSequenceClassification, BertTokenizer
from random import randint 
import pandas as pd

In [3]:
corpus = load_dataset('merionum/ru_paraphraser', data_files='plus.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-8471dca11b8f9d26/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
model = AutoModelForSequenceClassification.from_pretrained(model_name).cuda()
tokenizer = BertTokenizer.from_pretrained(model_name)

def compare_texts(text1, text2):
    batch = tokenizer(text1, text2, return_tensors='pt').to(model.device)
    with torch.inference_mode():
        proba = torch.softmax(model(**batch).logits, -1).cpu().numpy()
    return proba[0] # p(non-paraphrase), p(paraphrase)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [13]:
df_compare_texts = []

In [16]:
num_phrases = 3

for _ in range(num_phrases):
    randparaphrase = randint(0, corpus.shape['train'][0])

    phrase1 = corpus['train']['paraphrases'][randparaphrase][0]
    phrase2 = corpus['train']['paraphrases'][randparaphrase][1]

    df_compare_texts.append([phrase1, phrase2, compare_texts(phrase1, phrase2)])

In [10]:
pd.set_option('display.max_colwidth', None)

In [20]:
pd.DataFrame(df_compare_texts, columns=['phrase1', 'phrase2', 'p(non-paraphrase), p(paraphrase)'])

,phrase1,phrase2,"p(non-paraphrase), p(paraphrase)"
0,"""Герой дня"": Человек, бросивший яйцо в Януковича, стал замгубернатора на Украине","Украинец, бросавший яйцо в Януковича, стал заместителем губернатора","[0.032383967, 0.967616]"
1,Трамп возложил вину за лесные пожары в Калифорнии на власти штата,Трамп назвал виновных в лесных пожарах в Калифорнии,"[0.035794172, 0.9642058]"
2,"В районе вулкана Бардарбунга произошло землетрясение магнитудой 5,4 балла",В районе вулкана Бардарбунга произошло землетрясение,"[0.048926987, 0.95107305]"
